Import packages, load CSV files, and create copies of the datasets to clean them without corrupting the original data

In [ ]:
# Librairies

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Read csv
books_clean = pd.read_csv('books.csv')
ratings_clean = pd.read_csv('ratings.csv')
book_tags_clean = pd.read_csv('book_tags.csv')
tags_clean = pd.read_csv('tags.csv')
to_read_clean = pd.read_csv('to_read.csv')

Data Cleaning (Generic Process)

In [ ]:
# print("Affichage types données books.csv")
# print(books_clean.dtypes)

# print("Affichage type données ratings.csv")
# print(ratings_clean.dtypes)

# print("Affichage type données book_tags_clean.csv")
# print(book_tags_clean.dtypes)

# print("Affichage type données tags_clean.csv")
# print(tags_clean.dtypes)

# print("Affichage type données to_read_clean.csv")
# print(to_read_clean.dtypes)


# colonne_errone = []

# for colonne in ratings_clean.columns:
    # types = ratings_clean[colonne].apply(type).nunique()
    # if types > 1:
        # colonne_errone.append(colonne)
# print("Colonnes avec types errone ratings :", colonne_errone)

# for colonne in book_tags_clean.columns:
    # types = book_tags_clean[colonne].apply(type).nunique()
    # if types > 1:
        # colonne_errone.append(colonne)
# print("Colonnes avec types errone book_tags :", colonne_errone)

# for colonne in tags_clean.columns:
    # types = tags_clean[colonne].apply(type).nunique()
    # if types > 1:
        # colonne_errone.append(colonne)
# print("Colonnes avec types errone tags :", colonne_errone)

# for colonne in to_read_clean.columns:
    # types = to_read_clean[colonne].apply(type).nunique()
    # if types > 1:
        # colonne_errone.append(colonne)
# print("Colonnes avec types errone to_read :", colonne_errone)

# for colonne in books_clean.columns:
    # types = books_clean[colonne].apply(type).nunique()
    # if types > 1:
        # colonne_errone.append(colonne)        
# print("Colonnes avec types errone books :", colonne_errone)



# print("Vérification doublons")
# print(f"Doublons books_clean.csv : {books_clean.duplicated().sum()}")
# print(f"Doublons ratings_clean.csv : {ratings_clean.duplicated().sum()}")
# print(f"Doublons book_tags_clean.csv : {book_tags_clean.duplicated().sum()}")
# print(f"Doublons tags_clean.csv : {tags_clean.csv.duplicated().sum()}")
# print(f"Doublons to_read_clean.csv : {to_read_clean.csv.duplicated().sum()}")

# Delete duplicates
books_clean = books_clean.drop_duplicates()
ratings_clean = ratings_clean.drop_duplicates()
book_tags_clean = book_tags_clean.drop_duplicates()
tags_clean = tags_clean.drop_duplicates()
to_read_clean = to_read_clean.drop_duplicates()

# Delete spaces
books_clean.columns = books_clean.columns.str.strip().str.lower()
ratings_clean.columns = ratings_clean.columns.str.strip().str.lower()
book_tags_clean.columns = book_tags_clean.columns.str.strip().str.lower()
tags_clean.columns = tags_clean.columns.str.strip().str.lower()
to_read_clean.columns = to_read_clean.columns.str.strip().str.lower()


# print("Valeurs manquantes books_clean.csv")
# print(books_clean.isnull().sum())

# print("Valeurs manquantes ratings_clean.csv")
# print(ratings_clean.isnull().sum())

# print("Valeurs manquantes book_tags")
# print(book_tags_clean.isnull().sum())

# print("Valeurs manquantes tags_clean.csv")
# print(tags_clean.isnull().sum())

# print("Valeurs manquantes to_read_clean.csv")
# print(to_read_clean.isnull().sum())


# Delete missing values
books_clean = books_clean.dropna()
ratings_clean = ratings_clean.dropna()
book_tags_clean = book_tags_clean.dropna()
tags_clean = tags_clean.dropna()
to_read_clean = to_read_clean.dropna()

Data Cleaning (Specific Process)

In [ ]:
# Cleaning books

# Delete useless column
books_clean = books_clean.drop('isbn', axis=1)
books_clean = books_clean.drop('small_image_url', axis=1)
books_clean = books_clean.drop('work_ratings_count', axis=1)
books_clean = books_clean.drop('ratings_count', axis=1)

# Values over 0 for book_id, best_book_id, work_id and books_count
books_clean = books_clean[(books_clean['book_id'] > 0) & (books_clean['best_book_id'] > 0) & (books_clean['work_id'] > 0) & (books_clean['books_count'] > 0)]

# Values start with 978 or 979 and contains 13 numbers
books_clean = books_clean[(books_clean['isbn13'] // 10**10).isin([978, 979])]


In [ ]:
# Cleaning tags

# Values are over 0 for tag_id
tags_clean = tags_clean[tags_clean['tag_id'] > 0]

# Replace every "-" by " "
tags_clean['tag_name'] = tags_clean['tag_name'].str.replace(r'^-+', '', regex=True)
tags_clean['tag_name'] = tags_clean['tag_name'].str.replace(r'-+$', '', regex=True)
tags_clean['tag_name'] = tags_clean['tag_name'].str.replace(r'-+', ' ', regex=True)
tags_clean['tag_name'] = tags_clean['tag_name'].str.replace(r'\s+', ' ', regex=True).str.strip()


In [ ]:
# Cleaning book_tag

# Delete tag_id without match

# no_tags_id = book_tags_clean[~book_tags_clean['tag_id'].isin(tags_clean['tag_id'])]
# print("Valeurs sans correspondance tags_clean : ")
# print(no_tags_id['tag_id'].unique())
book_tags_clean = book_tags_clean[book_tags_clean['tag_id'].isin(tags_clean['tag_id'])]

# Delete book_id without match

# no_book_id = book_tags_clean[~book_tags_clean['book_id'].isin(books_clean['book_id'])]
# print("Valeurs sans correspondance books_clean : ")
# print(no_book_id['book_id'].unique())
book_tags_clean = book_tags_clean[book_tags_clean['goodreads_book_id'].isin(books_clean['book_id'])]

# Values are over 0 for tag_id, count and goodreads_book_id
book_tags_clean = book_tags_clean[(book_tags_clean['tag_id'] > 0) & (book_tags_clean['count'] > 0) & (book_tags_clean['goodreads_book_id'] > 0)]

In [ ]:
# Cleaning ratings

# Delete books_id without match

# no_id_books = ratings_clean[~ratings_clean['book_id'].isin(books_clean['book_id'])]
# print("Valeurs sans correspondance books_clean : ")
# print(no_id_books['book_id'].unique())
ratings_clean = ratings_clean[ratings_clean['book_id'].isin(books_clean['book_id'])]

# Values are between 1 and 5 for rating
ratings_clean = ratings_clean[(ratings_clean['rating'] > 0) & (ratings_clean['rating'] < 6)]

# Values over 0 for book_id and user_id
ratings_clean = ratings_clean[(ratings_clean['book_id'] > 0) & (ratings_clean['user_id'] > 0)]

In [ ]:
# Cleaning to_read

# Delete books_id without match

# no_books_id = to_read_clean[~to_read_clean['book_id'].isin(books_clean['book_id'])]
# print("Valeurs sans correspondance books_clean : ")
# print(no_books_id['book_id'].unique())
to_read_clean = to_read_clean[to_read_clean['book_id'].isin(books_clean['book_id'])]

# Values over 0 for book_id and user_id
to_read_clean = to_read_clean[(to_read_clean['book_id'] > 0) & (to_read_clean['user_id'] > 0)]

Create new files with every changement inside

In [ ]:
# Create clean csv
books_clean.to_csv('books_clean.csv', index=False)
ratings_clean.to_csv('ratings_clean.csv', index=False)
book_tags_clean.to_csv('book_tags_clean.csv', index=False)
tags_clean.to_csv('tags_clean.csv', index=False)
to_read_clean.to_csv('to_read_clean.csv')

Exploratory data analysis

In [ ]:
# Graphs tracking wishlist popularity and highest rated books

# Graphs tracking wishlist popularity and highest rated books
wishlist_counts = to_read_clean.merge(books_clean, on='book_id', how='left').groupby('title').size().reset_index(name='wishlist_count').sort_values(by='wishlist_count', ascending=False).head(10)

# Plot for most wished books
plt.figure(figsize=(10, 6))
sns.barplot(data=wishlist_counts, x='wishlist_count', y='title', palette='Blues_d')
plt.title('10 livres les plus ajouté à sa wishlist')
plt.xlabel('Nombre ajout')
plt.ylabel('Titre livre')
plt.tight_layout()
plt.show()

# Calculate total number of ratings per book
books_clean['total_ratings'] = books_clean['ratings_1'] + books_clean['ratings_2'] + books_clean['ratings_3'] + books_clean['ratings_4'] + books_clean['ratings_5']

# Keep only the 10 most rated books
books_clean = books_clean.sort_values(by='total_ratings', ascending=False).head(10)

# Display book titles and their total rating count
books_clean[['title', 'total_ratings']]


# Get the 10 highest rated books among the most rated ones
top_rated = books_clean.sort_values(by='average_rating', ascending=False).loc[:, ['title', 'average_rating']].head(10)

# Plot both most wishlisted and highest rated books side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

sns.barplot(data=wishlist_counts, x='wishlist_count', y='title', palette='Blues_d', ax=axes[0])
axes[0].set_title("Les 10 livres les plus ajoutés à la wishlist")
axes[0].set_xlabel("Nombre d'ajouts")
axes[0].set_ylabel("Titre du livre")

sns.barplot(data=top_rated, x='average_rating', y='title', palette='Oranges_d', ax=axes[1])
axes[1].set_title("Les 10 livres les mieux notés")
axes[1].set_xlabel("Note moyenne")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
# Graphs showing the average rating given by the top 10 readers

# Top 10 readers
top_users = ratings_clean.groupby('user_id').size().sort_values(ascending=False).head(10).index

# Ratings from the 10 readers
top_users_ratings = ratings_clean[ratings_clean['user_id'].isin(top_users)]

# Merge datas
top_users_ratings = top_users_ratings.merge(books_clean[['book_id', 'title']], on='book_id', how='left')

# Calculate the average rating given by each of the top 10 users
mean_ratings = top_users_ratings.groupby('user_id')['rating'].mean().reset_index()

# Plot the average ratings of the top 10 users as a bar chart
plt.figure(figsize=(10, 6))
sns.barplot(data=mean_ratings, x='user_id', y='rating', palette='Set2')

# Annotate each bar with the exact average rating rounded to one decimal place
for index, row in mean_ratings.iterrows():
    plt.text(x=index, y=row['rating'] + 0.05, s=round(row['rating'], 1), ha='center')

plt.title('Note moyenne des 10 plus gros lecteurs')
plt.xlabel('ID utilisateur')
plt.ylabel('Note moyenne')
plt.tight_layout()
plt.show()


# Graph showing the deviation of top readers' ratings from the average book ratings

# Calculate the average rating per book
average_book_ratings = ratings_clean.groupby('book_id')['rating'].mean().reset_index(name='average_rating')

# Merge average book ratings with all ratings
ratings_with_avg = ratings_clean.merge(average_book_ratings, on='book_id')

# Compute the difference between user rating and average book rating
ratings_with_avg['rating_diff'] = ratings_with_avg['rating'] - ratings_with_avg['average_rating']

# Identify the top 10 users by number of ratings
top_users = ratings_clean.groupby('user_id').size().sort_values(ascending=False).head(10).index

# Filter to keep only top users' ratings
top_users_ratings = ratings_with_avg[ratings_with_avg['user_id'].isin(top_users)]

# Calculate the mean rating difference per top user
mean_diff_per_user = top_users_ratings.groupby('user_id')['rating_diff'].mean().reset_index()

# Plot barplot of average rating difference per user
plt.figure(figsize=(10, 6))
sns.barplot(data=mean_diff_per_user, x='user_id', y='rating_diff', palette='coolwarm')

plt.title('Ecart des notes entre le top 10 des lecteurs et la note réelle du livre')
plt.xlabel('ID utilisateur')
plt.ylabel('Différence de notation (moyenne)')
plt.axhline(0, color='gray', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# Barplot displaying the top 20 most frequent tags in the book catalog

# Count the occurrences of each tag (assuming 'book_tags_clean' links books and tags)
tag_counts = book_tags_clean.groupby('tag_id').size().reset_index(name='count')

# Merge with tag names
tag_counts = tag_counts.merge(tags_clean[['tag_id', 'tag_name']], on='tag_id', how='left')

# Sort and keep the top 20 most frequent tags
top_tags = tag_counts.sort_values(by='count', ascending=False).head(20)

# Plot
plt.figure(figsize=(12, 8))
sns.barplot(data=top_tags, x='tag_name', y='count', palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Les 20 tags les plus courants')
plt.xlabel('Tags')
plt.ylabel('Nombre apparition')
plt.tight_layout()
plt.show()